## 実践演習13-3

scikit-learnのLabelPropagationでirisデータの半教師あり学習を行います。

ライブラリの読み込み

In [5]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading

irisデータの読み込み

In [2]:
iris = load_iris()
X = iris.data
y = iris.target

### 半教師ありデータの作成

まず、どのデータの教師信号を消すかを決めます。
[np.random.choice](https://docs.scipy.org/doc/numpy-dev/reference/generated/numpy.random.choice.html)を使って、データのインデックスを表すndarrayから、指定された個数のインデックスを、重複なしで抜き出します。

In [3]:
np.random.choice(np.arange(y.size), int((y.size)*.7), replace=False)

array([102,  51,  32,  22, 144,  29,  31,   6, 136, 138,  37,  80,  12,
        72,  43,  94,  56,  90, 119,  36,  69,   1, 130,  24,  83,  96,
         7,   5,  64,  19,  58, 105,  81,  54, 149, 146,  39,  60,  92,
        99,   8, 107,  61,  11,  26, 120,  84,   2,  21,  71,  28,   0,
        74,  75,   9, 118,  18,  45, 126, 123,  97,  95,  46,  66,  13,
       135, 139,  49, 111,  98,  41, 104, 114,  35,  47,  27, 132,  55,
        76,  15, 115,  68, 124,  40, 147, 109, 108,  85,  89,  93,  62,
       137,  77,  65, 113, 129, 131,  42, 122, 142, 103, 121, 116,  34,  59])

教師ベクトルyを[np.copy](https://docs.scipy.org/doc/numpy/reference/generated/numpy.copy.html)を使って変数labelsにコピーし（代入文でコピーすると、オブジェクトが共有されます）、先ほど抜き出したインデックスに対応する値を-1に書き換えます。

In [4]:
unlabeled_points = np.random.choice(np.arange(y.size), int((y.size)*.7), replace=False)
labels = np.copy(y)
labels[unlabeled_points] = -1
labels

array([-1,  0, -1, -1,  0, -1, -1, -1,  0, -1,  0, -1, -1,  0, -1, -1, -1,
       -1, -1, -1, -1, -1,  0,  0, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1,
        0, -1, -1,  0,  0, -1, -1,  0,  0, -1,  0, -1, -1, -1,  0, -1, -1,
       -1, -1, -1, -1, -1,  1,  1, -1,  1, -1, -1, -1,  1, -1,  1, -1, -1,
        1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        1,  1, -1, -1, -1,  1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,
       -1,  2, -1, -1,  2, -1, -1, -1,  2,  2, -1,  2,  2, -1, -1, -1, -1,
        2, -1, -1, -1,  2, -1, -1, -1, -1,  2,  2, -1, -1,  2,  2, -1,  2,
       -1,  2,  2,  2, -1,  2,  2, -1, -1, -1,  2,  2, -1, -1])

これを教師ベクトルとして、[LabelPropagation](http://scikit-learn.org/stable/modules/generated/sklearn.semi_supervised.LabelPropagation.html)で半教師あり学習を行います。

In [6]:
lp = LabelSpreading()
lp.fit(X, labels)

LabelSpreading(alpha=0.2, gamma=20, kernel='rbf', max_iter=30, n_jobs=1,
        n_neighbors=7, tol=0.001)

結果を表示します。

In [7]:
lp.score(X, y)

0.97333333333333338

正解付きデータを5%, 10%, 20%, 30%と変えて、それぞれ100回ずつ学習を試みて性能を評価します。

In [8]:
labeled_percent = [0.05, 0.1, 0.2, 0.3]
num = y.size
for labeled in labeled_percent :
    score = 0
    for i in range(100):
        unlabeled_points = np.random.choice(np.arange(num),int(num-num*labeled), replace=False)
        labels = np.copy(y)
        labels[unlabeled_points] = -1
        lp.fit(X, labels)
        score += lp.score(X, y)
    print("{0}{1:4.1f}{2}{3:6.3f}".format("labeled:", labeled*100, "%, score=", score/100))        

labeled: 5.0%, score= 0.872
labeled:10.0%, score= 0.931
labeled:20.0%, score= 0.956
labeled:30.0%, score= 0.964
